# Master Notebook with Combined Results

This notebook combines all relevant trial data and will apply statistical methods to the combined results. 

Currently it includes an overall $T$-$\omega_0$ plot. We are working on adding a linear model with metrics to this chart.

In [ ]:
import sys
sys.path.append('..')

from analyse.prelude import *

In [ ]:
trials, meta = load_files(['round2', 'round3', 'round4'])

for trial in trials:
    add_angular_momentum(trial, PRQ_I)

trials = filter_trials(trials, 1.1)
print('Number of filtered trials:', len(trials))

In [ ]:
first_row = []
last_row = []
for trial in trials:
    #print(trial.iloc[0])
    first_row.append(trial.iloc[0])
    last_row.append(trial.iloc[-1])
first_row = pd.DataFrame(first_row).reset_index(drop=True)
last_row  = pd.DataFrame(last_row).reset_index(drop=True)
L0 = np.sqrt(first_row['Lx']**2 + first_row['Ly']**2 + first_row['Lz']**2)
Lf = np.sqrt(last_row['Lx']**2 + last_row['Ly']**2 + last_row['Lz']**2)
delta_L = Lf - L0
#print(last_row.iloc[0])
delta_T = last_row['Time (s)'] - first_row['Time (s)']
average_torque = delta_L / delta_T
print(average_torque.head())
tau = average_torque.mean()
dtau = average_torque.std()
print('tau =', tau, '±', dtau)


In [ ]:
plot_trials_L([trials[24]], [meta[24]])

In [ ]:
period_data = point_trial_periods(trials, meta, plot_points=False)

plot_speed_vs_period(period_data)

fit_model(period_data)